In [1]:
import awkward as ak
import matplotlib.pyplot as plt
import os, sys
import subprocess
import json
import uproot
from coffea.nanoevents import NanoEventsFactory, ScoutingNanoAODSchema
from coffea.lookup_tools.lookup_base import lookup_base
import numpy as np
from coffea import processor, util
from hist import Hist
import hist
from coffea.analysis_tools import Weights, PackedSelection
from collections import defaultdict
import mplhep
plt.style.use(mplhep.style.CMS)

In [2]:
fileset = {
    "TTtoLNu2Q" : [
        "root://eosuser.cern.ch//eos/user/a/adlintul/TTtoLNu2Q_TuneCP5_13p6TeV_powheg-pythia8/Run3Summer22/230131_071621/0000/nanoaod_1.root"
    ],
#     "Run2022D" : [
#         "root://eosuser.cern.ch//eos/user/a/adlintul/ScoutingPFRun3/Run2022D/230206_163934/0000/scoutingnano_1.root"
#     ],
}

In [3]:
events = NanoEventsFactory.from_root(
    fileset["TTtoLNu2Q"][0],
    schemaclass=ScoutingNanoAODSchema,
).events()

In [4]:
fatjet = events.ScoutingFatJet

In [50]:
jets = events.ScoutingJet

In [5]:
muon = events.ScoutingMuon

In [48]:
goodmuon = (
            (events.ScoutingMuon.pt > 55)
            & (abs(events.ScoutingMuon.eta) < 2.4)
            & (abs(events.ScoutingMuon.trk_dxy) < 0.2)
            #& (abs(events.ScoutingMuon.trk_dz) < 0.5)
            #& (events.ScoutingMuon["type"] == 2)
            & (events.ScoutingMuon.normchi2 < 10)
            & (events.ScoutingMuon.nValidRecoMuonHits > 0)
            & (events.ScoutingMuon.nRecoMuonMatchedStations > 1)
            & (events.ScoutingMuon.nValidPixelHits > 0)
            & (events.ScoutingMuon.nTrackerLayersWithMeasurement > 5)            
        )

In [51]:
muons = events.ScoutingMuon[goodmuon]
near_jet, near_jet_dr = muons.nearest(jets, axis=1, return_metric=True)
muon_pt_rel = np.sqrt(muons.rho**2 - muons.dot(near_jet.unit)**2)

/opt/conda/lib/python3.8/site-packages/awkward/_connect/_numpy.py:195: RuntimeWarning: invalid value encountered in sqrt
  result = getattr(ufunc, method)(


In [58]:
(near_jet_dr > 0.4) | (muon_pt_rel > 25)

<Array [[], [], [], [], ... False], [], [], []] type='695 * var * ?bool'>

In [61]:
iso_muons = muons[(
    (near_jet_dr > 0.4) | (muon_pt_rel > 25)
)]

In [68]:
iso_muons

<PtEtaPhiMCollectionArray [[], [], [], [], ... [], [], [], []] type='695 * var *...'>

In [72]:
ak.num(iso_muons)

<Array [0, 0, 0, 0, 0, 0, ... 0, 0, 0, 0, 0, 0] type='695 * int64'>